In [4]:
import gradio as gr
import numpy as np
import pandas as pd
import joblib
from sklearn.base import BaseEstimator, RegressorMixin

# --- Define Ensemble Model class ---
class EnsembleModel(BaseEstimator, RegressorMixin):
    def __init__(self, model1, model2, weight1=0.5, weight2=0.5):
        self.model1 = model1
        self.model2 = model2
        self.weight1 = weight1
        self.weight2 = weight2

    def predict(self, X):
        return self.weight1 * self.model1.predict(X) + self.weight2 * self.model2.predict(X)

# --- Load Model and Scaler ---
best_pipe = joblib.load('ensemble_ann_rf_model.joblib')
y_scaler = joblib.load('y_scaler.joblib')

target_names = ["[Li]", "V of base", "[SO4]", "V of acid", "V of feed", "Voltage output"]

# --- Prediction + Performance Function ---
def predict_and_calculate(c_feed, flow_rate, current, time):
    input_sample = np.array([[c_feed, flow_rate, current, time]])
    predicted_scaled = best_pipe.predict(input_sample)
    predicted_outputs = y_scaler.inverse_transform(predicted_scaled)[0]
    pred = dict(zip(target_names, predicted_outputs))

    # Extract predictions and convert volume units
    Ct_li   = pred["[Li]"]
    Vt_base = pred["V of base"] / 1000
    Ct_so4  = pred["[SO4]"]
    Vt_acid = pred["V of acid"] / 1000
    Vt_feed = pred["V of feed"] / 1000

    # Constants
    F = 96485
    M_LiOH = 0.024
    M_H2SO4 = 0.098
    C0_Li = 0.1
    V0_Base = 0.5
    C0_SO4 = 0.05
    V0_Acid = 0.5
    V0_Feed = 0.5
    C0_Li_in_Feed = c_feed * 2
    C0_SO4_in_Feed = c_feed * 1

    # Simulate voltage profile for integration
    time_minutes = np.arange(0, time + 5, 5)
    time_seconds = time_minutes * 60
    sim_inputs = np.array([[c_feed, flow_rate, current, t] for t in time_minutes])
    sim_pred_scaled = best_pipe.predict(sim_inputs)
    sim_outputs = y_scaler.inverse_transform(sim_pred_scaled)
    voltage_profile = sim_outputs[:, target_names.index("Voltage output")]

    # Energy and recovery metrics
    integral_U_dt = np.trapezoid(voltage_profile, time_seconds)
    energy_joules = current * integral_U_dt
    energy_kWh = energy_joules / 3.6e6

    mol_Li = Ct_li * Vt_base - C0_Li * V0_Base
    mol_SO4 = Ct_so4 * Vt_acid - C0_SO4 * V0_Acid
    mass_LiOH = mol_Li * M_LiOH
    mass_H2SO4 = mol_SO4 * M_H2SO4
    sec_LiOH = energy_kWh / mass_LiOH if mass_LiOH > 0 else np.nan
    sec_H2SO4 = energy_kWh / mass_H2SO4 if mass_H2SO4 > 0 else np.nan

    rec_Li = mol_Li / (C0_Li_in_Feed * V0_Feed) * 100
    rec_SO4 = mol_SO4 / (C0_SO4_in_Feed * V0_Feed) * 100

    # Predicted output table (excluding voltage)
    pred_df = pd.DataFrame({
        "Predicted Output": ["[Li] (M)", "V of base (mL)", "[SO4] (M)", "V of acid (mL)", "V of feed (mL)"],
        "Value": [f"{pred['[Li]']:.3f}", f"{pred['V of base']:.3f}", f"{pred['[SO4]']:.3f}",
                  f"{pred['V of acid']:.3f}", f"{pred['V of feed']:.3f}"]
    })

    # Performance metrics table
    perf_df = pd.DataFrame({
        "Metric": [
            "SEC (LiOH, kWh/kg)", "SEC (H2SO4, kWh/kg)",
            "Recovery (Li⁺, %)", "Recovery (SO₄²⁻, %)"
        ],
        "Value": [
            f"{sec_LiOH:.2f}", f"{sec_H2SO4:.2f}",
            f"{rec_Li:.2f}", f"{rec_SO4:.2f}"
        ]
    })

    return pred_df, perf_df

# --- Gradio Interface ---
with gr.Blocks(title="ED GUI - Ensemble Model") as demo:
    gr.Markdown("<h2 style='font-size: 28px;'>Electrodialysis Output + Performance Estimator (ANN + RF Ensemble)</h2>")

    with gr.Row():
        c_feed = gr.Slider(0.5, 1.5, step=0.05, value=0.75, label="Feed Concentration (M)")
        flow_rate = gr.Slider(0.8, 1.2, step=0.1, value=1.0, label="Flow Rate (LPM)")
        current = gr.Slider(1.0, 2.0, step=0.05, value=1.5, label="Current (A)")
        time = gr.Slider(10, 180, step=5, value=60, label="Running Time (min)")

    with gr.Row():
        out_pred = gr.Dataframe(label="Predicted Outputs")
        out_perf = gr.Dataframe(label="Performance Metrics")

    run_button = gr.Button("Run Prediction")

    run_button.click(
        fn=predict_and_calculate,
        inputs=[c_feed, flow_rate, current, time],
        outputs=[out_pred, out_perf]
    )

demo.launch(inline=True)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
